# [HW10] Problem: Linear Methods on Fruits and Veggies

Import necessary Python packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import uniform
from numpy.random import randn
import random
import time
import matplotlib.pyplot as plt

from scipy.linalg import eig
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import svd

from utils import create_one_hot_label
from utils import subtract_mean_from_data
from utils import compute_covariance_matrix

import numpy as np
import numpy.linalg as LA

import sys
from numpy.linalg import svd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


### Visualize the image

**To start with this problem, we first take a look at a figure of banana in the dataset.**


In [ ]:
from IPython.display import Image
Image(filename='banana.png', width = 260, height = 260)


**Then we random shuffle the pixels in the above image.**


In [ ]:
# Read Images
img = mpimg.imread('banana.png')
rdmImg = np.reshape(img, (img.shape[0] * img.shape[1], img.shape[2]))
# shuffle
np.random.shuffle(rdmImg)
# reshape to original shape
rdmImg = np.reshape(rdmImg, img.shape)

plt.imshow(rdmImg)
plt.show()


**Let us visulize an image of an apple.**


In [ ]:
Image(filename='apple.png', width = 260, height = 260)


In [ ]:
# Read Images
img = mpimg.imread('apple.png')
rdmImg = np.reshape(img, (img.shape[0] * img.shape[1], img.shape[2]))
# shuffle
np.random.shuffle(rdmImg)
# reshape to original shape
rdmImg = np.reshape(rdmImg, img.shape)

plt.imshow(rdmImg)
plt.show()


**The above two images are exactly the same in terms of the HSV colorspace histogram feature.**


## Part (a). Random Projection.


In [ ]:
from projection import Projections, Project2D


In [ ]:
X = list(np.load('little_x_train.npy'))
Y = list(np.load('little_y_train.npy'))

CLASS_LABELS = ['apple', 'banana', 'eggplant']

feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)

rand_proj = projections.get_random_proj()
# Show Random 2D Projection
proj2D_viz = Project2D(rand_proj, CLASS_LABELS)
proj2D_viz.project_data(X, Y, white=np.eye(feat_dim), title='random projection')


## Part (b). PCA Projection.


In [ ]:
X = list(np.load('little_x_train.npy'))
Y = list(np.load('little_y_train.npy'))

CLASS_LABELS = ['apple', 'banana', 'eggplant']

feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)

# PCA Projection
pca_proj = projections.pca_projection(X, Y)

# Show PCA 2D Projection
proj2D_viz = Project2D(pca_proj, CLASS_LABELS)
proj2D_viz.project_data(X, Y, white=np.eye(feat_dim), title='PCA projection')


## Part (c). CCA Projection.


In [ ]:
X = list(np.load('little_x_train.npy'))
Y = list(np.load('little_y_train.npy'))

CLASS_LABELS = ['apple', 'banana', 'eggplant']

feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)

# CCA Projection
cca_proj, white_cov = projections.cca_projection(X, Y)
# Show CCA 2D Projection
proj2D_viz = Project2D(cca_proj, CLASS_LABELS)
proj2D_viz.project_data(X, Y, white=white_cov, title='CCA projection')


**Before we start the following parts, we first import several functions.**


In [ ]:
from projection import Project2D, Projections
from confusion_mat import getConfusionMatrixPlot

from ridge_model import Ridge_Model
from svm_model import SVM_Model
from logistic_model import Logistic_Model
from linear_classification import Model


### Part (d). Ridge Regression.


In [ ]:
# Load Training Data and Labels
X = list(np.load('little_x_train.npy'))
Y = list(np.load('little_y_train.npy'))

# Load Validation Data and Labels
X_val = list(np.load('little_x_val.npy'))
Y_val = list(np.load('little_y_val.npy'))

CLASS_LABELS = ['apple', 'banana', 'eggplant']

# Project Data to 200 Dimensions using CCA
feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)
cca_proj, white_cov = projections.cca_projection(X, Y, k=200)

X = projections.project(cca_proj, white_cov, X)
X_val = projections.project(cca_proj, white_cov, X_val)


#####RUN RIDGE REGRESSION#####
ridge_m = Ridge_Model(CLASS_LABELS)
model = Model(ridge_m)

model.train_model(X, Y)
model.test_model(X, Y)
model.test_model(X_val, Y_val)


### Part (e). LDA (Linear Discriminant Analysis).


In [ ]:
import numpy as np
import numpy.linalg as LA

import sys
from numpy.linalg import inv
from numpy.linalg import det
from sklearn.svm import LinearSVC
from projection import Project2D, Projections


class LDA_Model():
    def __init__(self,class_labels):
        self.lmbda = 0.001
        self.NUM_CLASSES = len(class_labels)
    def compute_class_means(self,X,Y):
        class_data = {}
        for i in range(len(X)):
            x = X[i]
            y = Y[i]
            if str(y) in class_data:
                class_data[str(y)].append(x)
            else:
                class_data[str(y)] = [x]
        self.class_means = {}

        for cls in class_data.keys():
            data = class_data[cls]
            data = np.array(data)
            mu = np.mean(data,0)
            self.class_means[cls] = mu

    def compute_class_cov(self,X,Y):
        N = len(X)
        x_dim = np.max(X[0].shape)
        C = np.zeros([x_dim,x_dim], dtype=float)

        # Calculate matrix C
        ### TODO: The code you need to fill is for calculating the covariance
        ###       matrix C for LDA.
        ### You need adding regularization in C (i.e., 0.001*np.eye(C.shape[0])).
        ### start code ###

        ### end code ###

        self.inv_cov =  inv(C)


    def train_model(self,X,Y):
        self.compute_class_means(X,Y)
        self.compute_class_cov(X,Y)


    def compute_likelihood(self,clss,x):
        inv_cov = self.inv_cov
        mean = self.class_means[clss]
        prod_term = -1*np.matmul((x-mean).T,np.matmul(inv_cov,(x-mean)))

        return prod_term



    def eval(self,x):
        prediction = np.zeros(self.NUM_CLASSES)
        i = 0
        for i in range(self.NUM_CLASSES):
            prediction[i] = self.compute_likelihood(str(i),x)
            i = i+1
        return np.argmax(prediction)

    def evals(self, X):
        # X has shape n*d
        X = np.array(X)
        N = X.shape[0]
        D = X.shape[1]
        C = self.NUM_CLASSES
        prediction = np.zeros((N, C))

        for i in range(C):
            mmean = X - np.reshape(self.class_means[str(i)], (1, -1))
            prediction[:, i] = -1 * np.ravel(np.reshape(mmean, (N, 1, D)) @ self.inv_cov @ np.reshape(mmean, (N, D, 1)))
        return np.argmax(prediction, axis=1)


#####RUN LDA REGRESSION#####

lda_m = LDA_Model(CLASS_LABELS)
model = Model(lda_m)

model.train_model(X, Y)
model.test_model(X, Y)
model.test_model(X_val, Y_val)


### Part (f). QDA (Quadratic Discriminant Analysis).


In [ ]:
class QDA_Model():

    def __init__(self, class_labels):

        self.lmbda = 0.001
        self.NUM_CLASSES = len(class_labels)

    def compute_class_means(self, X, Y):

        self.class_data = {}

        for i in range(len(X)):

            x = X[i]
            y = Y[i]

            if str(y) in self.class_data:
                self.class_data[str(y)].append(x)
            else:
                self.class_data[str(y)] = [x]

        self.class_means = {}

        for cls in self.class_data.keys():
            data = self.class_data[cls]

            data_n = np.array(data)
            mu = np.mean(data_n, 0)

            self.class_means[cls] = mu

    def compute_class_cov(self, X, Y):

        N = len(X)

        x_dim = np.max(X[0].shape)

        self.class_inv_cov = {}
        self.class_cov = {}

        for cls in self.class_data.keys():
            C = np.zeros([x_dim, x_dim], dtype=float)
            data = self.class_data[cls]
            mu = self.class_means[cls]
            N = len(data)

            # Calculate matrix C
            ### TODO: The code you need to fill is for calculating the covariance
            ###       matrix C for QDA.
            ### You need adding regularization in each C (i.e., 0.001*np.eye(C.shape[0])).
            ### start code ###

            ### end code ###


            self.class_inv_cov[cls] = inv(C)
            self.class_cov[cls] = np.copy(C)

    def train_model(self, X, Y):

        self.compute_class_means(X, Y)
        self.compute_class_cov(X, Y)

    def compute_likelihood(self, clss, x):

        cov = self.class_cov[clss]
        inv_cov = self.class_inv_cov[clss]
        mean = self.class_means[clss]
        prod_term = -np.log(det(cov)) - np.matmul((x - mean).T, np.matmul(inv_cov, (x - mean)))

        return prod_term

    def eval(self, x):

        prediction = np.zeros(self.NUM_CLASSES)
        i = 0
        for i in range(self.NUM_CLASSES):
            prediction[i] = self.compute_likelihood(str(i), x)
            i = i + 1

        return np.argmax(prediction)


#####RUN QDA REGRESSION#####

qda_m = QDA_Model(CLASS_LABELS)
model = Model(qda_m)

model.train_model(X, Y)
model.test_model(X, Y)
model.test_model(X_val, Y_val)


### Part (g). Linear SVM.


In [ ]:
#####RUN SVM REGRESSION#####

svm_m = SVM_Model(CLASS_LABELS)
model = Model(svm_m)

model.train_model(X, Y)
model.test_model(X, Y)
model.test_model(X_val, Y_val)


### Part (h). Logistic Regression


In [ ]:
#####RUN Logistic REGRESSION#####
lr_m = Logistic_Model(CLASS_LABELS)
model = Model(lr_m)

model.train_model(X, Y)
model.test_model(X, Y)
model.test_model(X_val, Y_val)


### Part (i)-(j). ROC (Receiver Operating Characteristic).


In [ ]:
CLASS_LABELS = ['apple', 'banana']


def compute_tp_fp(thres, scores, labels):
    scores = np.array(scores)
    prediction = (scores > thres)
    tp = np.sum(prediction * labels)
    tpr = 1.0 * tp / np.sum(labels)

    fp = np.sum(prediction * (1 - labels))
    fpr = 1.0 * fp / np.sum(1 - labels)
    return tpr, fpr


def plot_ROC(tps, fps):
    # plot
    plt.plot(fps, tps)
    plt.ylabel("True Positive Rates")
    plt.xlabel("False Positive Rates")


def ROC(scores, labels):
    thresholds = sorted(np.unique(scores))
    thresholds = [-float("Inf")] + thresholds + [float("Inf")]
    tps = []
    fps = []

    ### TODO: The code you need to fill in is to apply the 'compute_tp_fp' function to
    ###       compute the true positive rate (TPR) and false positive rate (FPR),
    ###       and add the calculated tpr and fpr to the lists: tps and fps.
    ### start code ###

    ### end code ###

    return tps, fps

def eval_with_ROC(method, train_X, train_Y, val_X, val_Y, C, change_score=False):
    m = method(CLASS_LABELS)
    m.C = C
    m.train_model(train_X, train_Y)
    scores = m.scores(val_X)
    #### change the scores here
    if change_score:
        scores = 10.0 * np.array(scores)

    tps, fps = ROC(scores, val_Y)
    plot_ROC(tps, fps)

def trim_data(X, Y):
    # throw away the 3rd class data
    X = np.array(X)
    Y = np.array(Y)
    retain = (Y < 2)
    return X[retain, :], Y[retain]


# Load Training Data and Labels
X = list(np.load('little_x_train.npy'))
Y = list(np.load('little_y_train.npy'))
X, Y = trim_data(X, Y)

# Load Validation Data and Labels
X_val = list(np.load('little_x_val.npy'))
Y_val = list(np.load('little_y_val.npy'))
X_val, Y_val = trim_data(X_val, Y_val)

CLASS_LABELS = ['apple', 'banana']

# Project Data to 200 Dimensions using CCA
feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)
cca_proj, white_cov = projections.cca_projection(X, Y, k=200)
X = projections.project(cca_proj, white_cov, X)
X_val = projections.project(cca_proj, white_cov, X_val)

##### RUN SVM REGRESSION #####
eval_with_ROC(SVM_Model, X, Y, X_val, Y_val, 1.0)
eval_with_ROC(SVM_Model, X, Y, X_val, Y_val, 0.01)
plt.legend(["C=1.0", "C=0.01"])
plt.show()

##### RUN SVM REGRESSION (after changing score) #####
eval_with_ROC(SVM_Model, X, Y, X_val, Y_val, 1.0, change_score=True)
eval_with_ROC(SVM_Model, X, Y, X_val, Y_val, 1.0, change_score=False)
plt.legend(["C=0.01", "C=0.01 (multiply the scores)"])
plt.show()


### Part (k). Hyperparameters Search.


In [ ]:
from confusion_mat import getConfusionMatrix
from confusion_mat import plotConfusionMatrix


In [ ]:
CLASS_LABELS = ['apple', 'banana', 'nectarine', 'plum', 'peach', 'watermelon', 'pear', 'mango', 'grape', 'orange',
                'strawberry', 'pineapple',
                'radish', 'carrot', 'potato', 'tomato', 'bellpepper', 'broccoli', 'cabbage', 'cauliflower', 'celery',
                'eggplant', 'garlic', 'spinach', 'ginger']


def eval_model(X, Y, k, model_key, proj):
    # PROJECT DATA
    cca_proj, white_cov = proj.cca_projection(X, Y, k=k)

    X_p = proj.project(cca_proj, white_cov, X)
    X_val_p = proj.project(cca_proj, white_cov, X_val)

    # TRAIN MODEL
    model = models[model_key]

    model.train_model(X_p, Y)
    acc, cm = model.test_model(X_val_p, Y_val)

    return acc, cm


class Model():
    """ Generic wrapper for specific model instance. """

    def __init__(self, model):
        """ Store specific pre-initialized model instance. """

        self.model = model

    def train_model(self, X, Y):
        """ Train using specific model's training function. """

        self.model.train_model(X, Y)

    def test_model(self, X, Y):
        """ Test using specific model's eval function. """
        if hasattr(self.model, "evals"):
            labels = np.array(Y)
            p_labels = self.model.evals(X)
            success = np.sum(labels == p_labels)
            total_count = len(X)

        else:
            labels = []  # List of actual labels
            p_labels = []  # List of model's predictions
            success = 0  # Number of correct predictions
            total_count = 0  # Number of images

            for i in range(len(X)):

                x = X[i]  # Test input
                y = Y[i]  # Actual label
                y_ = self.model.eval(x)  # Model's prediction
                labels.append(y)
                p_labels.append(y_)

                if y == y_:
                    success += 1
                total_count += 1

        return 1.0 * success / total_count, getConfusionMatrix(labels, p_labels)




# Load Training Data and Labels
X = list(np.load('big_x_train.npy'))
Y = list(np.load('big_y_train.npy'))

# Load Validation Data and Labels
X_val = list(np.load('big_x_val.npy'))
Y_val = list(np.load('big_y_val.npy'))

# Project Data to 200 Dimensions using CCA
feat_dim = max(X[0].shape)
projections = Projections(feat_dim, CLASS_LABELS)

models = {}  # Dictionary of key: model names, value: model instance

#########MODELS TO EVALUATE############
qda_m = QDA_Model(CLASS_LABELS)
models['qda'] = Model(qda_m)

lda_m = LDA_Model(CLASS_LABELS)
models['lda'] = Model(lda_m)

ridge_m = Ridge_Model(CLASS_LABELS)
models['ridge'] = Model(ridge_m)

ridge_m_10 = Ridge_Model(CLASS_LABELS)
ridge_m_10.lmbda = 10.0
models['ridge_lmda_10'] = Model(ridge_m_10)

ridge_m_01 = Ridge_Model(CLASS_LABELS)
ridge_m_01.lmbda = 0.1
models['ridge_lmda_01'] = Model(ridge_m_01)

# svm_m = SVM_Model(CLASS_LABELS)
# models['svm'] = Model(svm_m)

svm_m_01 = SVM_Model(CLASS_LABELS)
svm_m_01.C = 0.1
models['svm_C_01'] = Model(svm_m_01)

svm_m_001 = SVM_Model(CLASS_LABELS)
svm_m_001.C = 0.01
models['svm_C_001'] = Model(svm_m_001)

#########GRID SEARCH OVER MODELS############
highest_accuracy = 0  # Highest validation accuracy
best_model_name = None  # Best model name
best_model = None  # Best model instance
best_k = None

K = [20, 25, 30, 50, 100]  # List of dimensions

for model_key in models.keys():
    print(model_key)

    val_acc = []  # List of model's accuracies for each dimension
    for k in K:
        print("k =", k)

        # Evaluate specific model's validation accuracy on specific dimension
        acc, c_m = eval_model(X, Y, k, model_key, projections)

        val_acc.append(acc)

        if acc > highest_accuracy:
            highest_accuracy = acc
            best_model_name = model_key
            best_cm = c_m
            best_k = k

    # Plot specific model's accuracies across validation error
    plt.plot(K, val_acc, label=model_key)

# Display aggregate plot of models across validation error
plt.legend()
plt.xlabel('Dimension')
plt.ylabel('Accuracy')
plt.show()

# Plot best model's confusion matrix
plotConfusionMatrix(best_cm, CLASS_LABELS)


In [ ]:
print('best model: ', best_model_name)
print('best k: ', best_k)


**You could try higher dimensions for CCA, also could try to use larger $C$ in SVM, but it may take more than one hour to run the above hyper parameter search part.**
